In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
pip install beautifulsoup4 #BeautifulSoup for converting HTML into Dataframe

     |████████████████████████████████| 122kB 4.8MB/s eta 0:00:01     |███████████▎                    | 40kB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

     |████████████████████████████████| 5.5MB 6.9MB/s eta 0:00:01��██████                       | 1.6MB 6.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

postal_codes_dict = {} # initialize an empty dictionary to save the data in
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text # get the postal code
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text # get the rest of the data in the cell
        borough = neighborhoods_data.split('(')[0] # get the borough in the cell
        
        # if the cell is not assigned then ignore it
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        # else process the data and add it to the dictionary
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                # remove parantheses from neighborhoods string
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough
 
            # add borough and neighborhood to dictionary
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass
    
# create an empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

# print number of rows of dataframe
toronto_data.shape[0]

0

In [5]:
toronto_table=soup.find("table", attrs={"class":"wikitable sortable"})
toronto_table_data= toronto_table.tbody.find_all("tr")


In [6]:
#headings of the wikipedia table
table_heading=[]
for th in toronto_table.find_all("th"):
    table_heading.append(th.text.replace('\n','').strip())
#rows in the wikipedia table
table_rows =[]
for tr in toronto_table.find_all("tr"):
    each_row={}
    for td, th in zip(tr.find_all("td"), table_heading):
         each_row[th] = td.text.replace('\n','').strip()
    table_rows.append(each_row)
        
table_df=pd.DataFrame(table_rows[1:],columns=table_heading)
#table_df


In [7]:
#select rows that have the assigned borough

table_df2=table_df[table_df.Borough != 'Not assigned']
#assign Borough name to the 'not-assigned' neighborhood
for row in range(len(table_df2)): 
    if table_df2.iloc[row,2] =='Not assigned':  
        table_df2.iloc[row,2] == table_df2.iloc[row,1]
table_df2.reset_index(drop=True, inplace=True)
table_df2



,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
table_df2.shape[0]

103

In [9]:
import os
os.getcwd() 
os.chdir('/resources/data')  #.csv file is uploaded into this folder

df_lat_long=pd.read_csv('Geospatial_Coordinates.csv')
#print(df_lat_long)  # The .csv file for postal codes with latitudes and longitudes

#Add two more columns for Latitude and Longitude
table_df2['Latitude'] = None 
table_df2['Longitude']= None
# Sort the Latitudes and Longitudes before assigning them to the postal codes for neighborhoods
for row in range(len(table_df2)):
    for temp_row in range(len(df_lat_long)):
        if table_df2['Postal Code'].iloc[row]==df_lat_long['Postal Code'].iloc[temp_row]:
            table_df2['Latitude'].iloc[row]  =df_lat_long['Latitude'].iloc[temp_row]
            table_df2['Longitude'].iloc[row] =df_lat_long['Longitude'].iloc[temp_row]
                 
table_df2  # Neighborhood in Toronto with postal codes and Latitude and Longitude information

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6679,-79.5322
6,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
7,M3B,North York,Don Mills,43.7459,-79.3522
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789


In [10]:
neighborhood_toronto = folium.Map(location=[table_df2['Latitude'].iloc[3], table_df2['Longitude'].iloc[3]], zoom_start=10)
#neighborhood_toronto

# add markers to map
for label, lat, lng in zip( table_df2['Neighbourhood'], table_df2['Latitude'], table_df2['Longitude']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(neighborhood_toronto)  
    
neighborhood_toronto

In [11]:
#Use KMeans to cluster the neighborhoods in Toronto

neighborhood_toronto_clusters=table_df2 #Create a new dataframe to which cluster labels will be added
neighborhood_clusters = 5
neighborhood_toronto_clustering = neighborhood_toronto_clusters[['Latitude','Longitude']]
kmeans = KMeans(n_clusters = neighborhood_clusters, random_state = 0).fit(neighborhood_toronto_clustering)
cluster_labels= kmeans.labels_

neighborhood_toronto_clusters['Cluster Label']=cluster_labels #Add cluster label column to the main dataframe


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [12]:
# create map of neighborhood clusters in Toronto

# map initialization was carried out earlier as "neighborhood_toronto"

# set color scheme for different clusters
x = np.arange(neighborhood_clusters)
ys = [i + x + (i*x)**2 for i in range(neighborhood_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map for different clusters

markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_toronto_clusters['Latitude'], neighborhood_toronto_clusters['Longitude'], neighborhood_toronto_clusters['Neighbourhood'], neighborhood_toronto_clusters['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(neighborhood_toronto)
       
neighborhood_toronto